<a href="https://colab.research.google.com/github/danielphil/fastai-sandbox/blob/master/lesson1/miffy_or_kitty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will attempt to predict if an image contains Miffy or Hello Kitty. It uses your webcam, so no need to mess around with uploading images.

No idea why I picked these subjects for a toy deep learning project, but I guess they look a little similar, so I thought it might be an interesting challenge for the network. 😉

This was the result of following Lesson 1 of fast.ai's [Practical Deep Learning for Coders](https://course.fast.ai/) course.

To run:
* Select `Runtime->Run all` from the menu above.
* Scroll to the bottom of the page. You should see a live display of your webcam.
* Show some Miffy or Hello Kitty related images to the camera and the model will show the probabilities it thinks of your image being either Hello Kitty or Miffy.

In [0]:
# colab setup
!curl -s https://course.fast.ai/setup/colab | bash
# download model
!curl -o /content/models/export.pkl https://raw.githubusercontent.com/danielphil/fastai-sandbox/master/lesson1/export.pkl

from fastai.vision import *
defaults.device = torch.device('cpu')

learn = load_learner('/content/models/')

import IPython
from google.colab import output
from base64 import b64decode

def Predict(b64_data):
  filename = 'photo.jpg'
  binary = b64decode(b64_data.split(',')[1])

  with open(filename, 'wb') as f:
    f.write(binary)

  pred_class,pred_idx,outputs = learn.predict(open_image(filename))
  result = ""
  for i in range(len(outputs)):
    result += '{}: {:.2f}%\n'.format(learn.data.classes[i], outputs[i] * 100)

  return IPython.display.JSON({'result': result})

output.register_callback('notebook.Predict', Predict)

In [0]:
%%javascript
const timeDelayMs = 1000

async function captureAndPredict(textarea, video, status) {
  const canvas = document.createElement('canvas');
  canvas.width = video.videoWidth;
  canvas.height = video.videoHeight;
  canvas.getContext('2d').drawImage(video, 0, 0);
  imageData = canvas.toDataURL('image/jpeg', 0.8);

  const result = await google.colab.kernel.invokeFunction(
    'notebook.Predict', // The callback name.
    [imageData], // The arguments.
    {}); // kwargs

  textarea.value = result.data['application/json'].result;

  if (!status.done) {
    setTimeout(() => captureAndPredict(textarea, video, status), timeDelayMs);
  }
}

async function takePhoto(div) {
  const stop = document.createElement('button');
  const text = document.createElement('textarea');
  text.textContent = "Please wait..."

  stop.textContent = 'Stop';
  div.appendChild(stop);
  div.appendChild(text);

  const video = document.createElement('video');
  video.style.display = 'block';
  const stream = await navigator.mediaDevices.getUserMedia({video: true});

  document.body.appendChild(div);
  div.appendChild(video);
  video.srcObject = stream;
  await video.play();

  // Resize the output to fit the video element.
  google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

  var status = { done: false };

  setTimeout(() => captureAndPredict(text, video, status), timeDelayMs)

  // Wait for Done to be clicked.
  await new Promise((resolve) => stop.onclick = resolve);

  status.done = true;
  stream.getVideoTracks()[0].stop();
  div.remove();
}

(async function() {
  const div = document.createElement('div');
  document.querySelector("#output-area").appendChild(div);
  await takePhoto(div);
})();